# Deepfake Image Detection

Autori: Bucă Mihnea-Vicențiu; Căpatână Răzvan-Nicolae; Luculescu Teodor


## Cross-generator deepfake detection

We want to evaluate the generalization capabilities of deepfake detection methods: how well detectors work when tested on images produced by other generators than those seen at training. For this, we will train on images coming from one generator and test on images coming from other generators. We will compare at least three different methods.

First method:
- We will use an image classification architecture, mainly ResNet, that will be trained from scratch.

Second method:
- The same architecture as above, but this time initialized with pre-trained weights. The weights will be obtained by supervised learning (image classification on ImageNet).


Third method:
- Large pretrained self-supervised representations followed by a linear classifier. In this case, only the linear classifier is trained; the representations are extracted with a frozen model. [Ojha et al., (2023)](https://github.com/WisconsinAIVision/UniversalFakeDetect) have used this approach in the context of deepfake detection, but differently from us, they have applied it to general fully-generated images. We will train two models, one using CLIP and one using SAM self-supervised representations and compare the results obtained.


For each method, fill in a table with the average precisions.

In this notebook we will use the **Second method**

### Data

The dataset can be downloaded from [here](https://drive.google.com/file/d/1NfLX9bZtOY8dO_yj3cU7pEHGmqItqjg2/view). It contains real images from the CelebAHQ dataset and locally manipulated images produced by four generators: [LDM](https://github.com/CompVis/latent-diffusion), [Pluralistic](https://github.com/lyndonzheng/Pluralistic-Inpainting), [LAMA](https://github.com/advimman/lama), [Repaint](https://github.com/andreas128/RePaint). You can read more about how this dataset was produced in Section 3.3 of the following paper:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will extract the data for each model

In [ ]:
import zipfile
import os

# path to the zip file
zip_file_path = 'drive/MyDrive/Proiect DeepLearning/DeepFMI_local_data.zip'

# the paths to the datasets within the zip file
dataset_paths = [
    'FMI_local_data/celebhq_real_data',
    'FMI_local_data/lama',
    'FMI_local_data/ldm',
    'FMI_local_data/pluralistic',
    'FMI_local_data/repaint'
]

# create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Iterate through the dataset paths
    for dataset_path in dataset_paths:
         zip_ref.extractall(members=[
            name for name in zip_ref.namelist()
            if name.startswith(dataset_path)
        ], path='/content/')  # Extract to the '/content/' directory


## Training Models

!pip install sam

In [ ]:
# important libraries
import torch
import glob
import pandas as pd
import torch.nn as nn
import torch.optim as optim

import numpy as np

import timm
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import display, Markdown
from sklearn.metrics import average_precision_score
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torchvision.models as models
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from PIL import Image
from tqdm import tqdm  # for progress bar

In [ ]:
class DeepFakeDataset(Dataset):
    """
    Takes two folders (real vs fake) and assigns labels 0 / 1.
    """
    def __init__(self, real_folder: str, fake_folder: str, transform=None):
        # grab all .png under each
        self.real_paths = sorted(glob.glob(os.path.join(real_folder, '*.png')))
        self.fake_paths = sorted(glob.glob(os.path.join(fake_folder, '*.png')))

        # create a single list of (path, label)
        # real = 0, fake = 1
        self.samples = (
            [(p, 0) for p in self.real_paths] +
            [(p, 1) for p in self.fake_paths]
        )
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

In [ ]:
def make_model_dataloaders(
    root_dir: str,            # contains subfolders: lama/, ldm/, repaint/, pluralistic/
    real_root: str,           # path to celebhq_real_data
    model_names: list[str],   # ['lama','ldm','repaint','pluralistic']
    splits: list[str] = ('train','valid','test'),
    batch_size: int = 16,
    img_size: int = 256,
    num_workers: int = 2
):
    """
    Returns a dict:
      { model_name: { split: DataLoader, … }, … }
      Each loader mixes real vs that model's fake images.
    """

    # strong augmentation for train
    train_tf = transforms.Compose([
        transforms.RandomResizedCrop(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize((0.4914,0.4822,0.4465), (0.2023,0.1994,0.2010))
    ])

    # weak augmentation for val/test
    test_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914,0.4822,0.4465), (0.2023,0.1994,0.2010))
    ])

    dataloaders = {}
    for model_name in model_names:
            dataloaders[model_name] = {}
            for split in splits:
                real_folder = os.path.join(real_root, split)
                fake_folder = os.path.join(root_dir, model_name, split)

                tf = train_tf if split=='train' else test_tf
                ds = DeepFakeDataset(real_folder, fake_folder, transform=tf)

                dataloaders[model_name][split] = DataLoader(
                    ds,
                    batch_size=batch_size,
                    shuffle=(split=='train'),
                    num_workers=num_workers,
                    pin_memory=True
                )

    return dataloaders

In [ ]:
root = "/content/FMI_local_data"
deepfake_models = ["lama", "ldm", "repaint", "pluralistic"]
loaders = make_model_dataloaders(
    root_dir=root,
    real_root=os.path.join(root, "celebhq_real_data"),
    model_names=deepfake_models,
    splits=['train', 'valid', 'test'],
    batch_size=16,
    img_size=256,
    num_workers=2
)

# test
ldm_train_loader = loaders['ldm']['train']
print(f"ldm train batches: {len(ldm_train_loader)}")

ldm train batches: 1125


In [ ]:
def train_timm(
    dataloaders,         # dict: { name: {'train','valid','test'} DataLoaders }
    num_classes: int = 2,
    num_epochs: int = 2,
    lr_head: float = 1e-3,
    lr_ft: float = 1e-4,
    weight_decay: float = 1e-5,
    freeze_epochs: int = 5,
    device: str = None
):
    """
    Fine-tune Xception41 pretrained on ImageNet-1K (supervised)
    - Phase 1: freeze backbone, train only the new classifier head
    - Phase 2: unfreeze entire network and fine-tune
    """
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    results = {}

    for name, splits in dataloaders.items():
        print(f"\n=== [Xception-Imagenet] Training on '{name}' ===")

        # 1) Load Xception41 with supervised ImageNet‐1K weights
        model = timm.create_model(
            'xception41',
            pretrained='imagenet',
            num_classes=num_classes
        ).to(device)

        # 2) Phase 1: freeze all layers except the classifier head
        for p in model.parameters():
            p.requires_grad = False
        # timm’s get_classifier() returns the final Linear
        for p in model.get_classifier().parameters():
            p.requires_grad = True

        optimizer = optim.Adam(
            model.get_classifier().parameters(),
            lr=lr_head,
            weight_decay=weight_decay
        )
        scheduler = optim.lr_scheduler.StepLR(
            optimizer, step_size=freeze_epochs, gamma=0.1
        )
        criterion = nn.CrossEntropyLoss()

        # Head-only training
        for epoch in range(1, freeze_epochs + 1):
            model.train()
            running_loss = 0.0
            for imgs, labels in splits['train']:
                imgs, labels = imgs.to(device), labels.to(device)
                optimizer.zero_grad()
                logits = model(imgs)             # raw logits
                loss = criterion(logits, labels) # CE on logits
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * imgs.size(0)
            scheduler.step()
            avg = running_loss / len(splits['train'].dataset)
            print(f" [Head] Epoch {epoch}/{freeze_epochs} — loss: {avg:.4f}")

        # 3) Phase 2: unfreeze everything, fine-tune
        for p in model.parameters():
            p.requires_grad = True

        optimizer = optim.Adam(
            model.parameters(),
            lr=lr_ft,
            weight_decay=weight_decay
        )
        scheduler = optim.lr_scheduler.StepLR(
            optimizer, step_size=5, gamma=0.1
        )

        for epoch in range(freeze_epochs + 1, num_epochs + 1):
            model.train()
            running_loss = 0.0
            for imgs, labels in splits['train']:
                imgs, labels = imgs.to(device), labels.to(device)
                optimizer.zero_grad()
                logits = model(imgs)
                loss = criterion(logits, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * imgs.size(0)
            scheduler.step()
            avg = running_loss / len(splits['train'].dataset)
            print(f" [Fine-tune] Epoch {epoch}/{num_epochs} — loss: {avg:.4f}")

        # 4) Validation
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for imgs, labels in splits['valid']:
                imgs, labels = imgs.to(device), labels.to(device)
                preds = model(imgs).argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        valid_acc = correct / total
        print(f" ▶ valid acc {valid_acc:.4%}")


        results[name] = {
            'model': model,
            'valid_acc': valid_acc,
        }

        # clear GPU memory
        torch.cuda.empty_cache()

    return results

In [ ]:
results = train_timm(
    dataloaders=loaders,
    num_classes = 2,
    num_epochs = 2,
    lr_head = 1e-3,
    lr_ft = 1e-4,
    weight_decay = 1e-5,
    freeze_epochs = 1,
)


=== [Xception-Imagenet] Training on 'lama' ===
 [Head] Epoch 1/1 — loss: 0.4419
 [Fine-tune] Epoch 2/2 — loss: 0.0921
 ▶ valid acc 98.5000%

=== [Xception-Imagenet] Training on 'ldm' ===
 [Head] Epoch 1/1 — loss: 0.5636
 [Fine-tune] Epoch 2/2 — loss: 0.1161
 ▶ valid acc 95.5000%

=== [Xception-Imagenet] Training on 'repaint' ===
 [Head] Epoch 1/1 — loss: 0.7082
 [Fine-tune] Epoch 2/2 — loss: 0.6140
 ▶ valid acc 74.3889%

=== [Xception-Imagenet] Training on 'pluralistic' ===
 [Head] Epoch 1/1 — loss: 0.6563
 [Fine-tune] Epoch 2/2 — loss: 0.3781
 ▶ valid acc 77.9444%


In [ ]:
import os
# save models in drive/MyDrive/Proiect DeepLearning/Second-Method
save_path = 'drive/MyDrive/Proiect DeepLearning/Second-Method/results.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Ensure the directory exists
torch.save(results, save_path)
print(f"Results saved to: {save_path}")

NameError: name 'os' is not defined

In [ ]:
def build_cross_table_ap(results: dict, dataloaders: dict, device=None) -> pd.DataFrame:
    """
    Returns a cross-table with average precision score (macro-averaged).
    Assumes classification with multiple classes.
    """
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    gens = list(results.keys())
    table = pd.DataFrame(index=gens, columns=gens, dtype=float)

    for train_gen, result_dict in results.items():
        model = result_dict['model']
        model.to(device).eval()

        with torch.no_grad():
            for test_gen in gens:
                all_probs = []
                all_targets = []
                for imgs, labels in dataloaders[test_gen]['test']:
                    imgs = imgs.to(device)
                    logits = model(imgs)  # raw outputs
                    probs = torch.softmax(logits, dim=1).cpu().numpy()
                    all_probs.append(probs)
                    all_targets.append(labels.cpu().numpy())

                all_probs = np.concatenate(all_probs, axis=0)
                all_targets = np.concatenate(all_targets, axis=0)

                # Convert labels to one-hot
                num_classes = all_probs.shape[1]
                targets_onehot = np.eye(num_classes)[all_targets]

                ap_score = average_precision_score(targets_onehot, all_probs, average='macro')
                table.loc[test_gen, train_gen] = ap_score

    return table

In [ ]:
# torch.serialization.add_safe_globals([timm.models.xception_aligned.XceptionAligned])
# tmp = torch.load('drive/MyDrive/Proiect DeepLearning/Second-Method/results.pth', weights_only=False)
df = build_cross_table_ap(results, loaders)
print("\nCross-Generator Accuracy Table:\n")
display(Markdown(df.to_markdown(floatfmt=".3f")))


Cross-Generator Accuracy Table:



|             |   lama |   ldm |   repaint |   pluralistic |
|:------------|-------:|------:|----------:|--------------:|
| lama        |  0.998 | 0.310 |     0.445 |         0.626 |
| ldm         |  0.410 | 0.993 |     0.906 |         0.460 |
| repaint     |  0.486 | 0.636 |     0.800 |         0.531 |
| pluralistic |  0.613 | 0.421 |     0.586 |         0.799 |